# Link prediction example using the GNN learning engine under the RelationalAI Snowflake Native App

## ✅ Prerequisites

Before running this notebook, make sure you've completed the following setup steps:

1. **Install the RelationalAI Experimental Application**
   Ensure that the RelationalAI experimental app is installed in your Snowflake account.

2. **Preprocess the Tokenflow Dataset**
   Run the `preprocess_tokenflow_data.ipynb` notebook to generate the processed dataset required for this tutorial.

3. **Upload the Preprocessed Data to Snowflake**
   Use the preprocessing notebook to upload the Tokenflow data to the appropriate tables in your Snowflake environment.

4. **Install the RelationalAI GNN Python SDK**
   Follow the installation steps provided in the [README.md](GNN_SDK_DOCUMENTATION.md) to set up the RelationalAI GNN SDK in your environment.

### 🔐 Connecting to the RelationalAI Native App via Snowflake

To connect to the RelationalAI native app, we'll first set up a dictionary that defines the necessary environment variables for establishing a Snowflake connection. Here we will use
the `active_session` authentication method which gets all the credentials needed from the
current snowflake session. We only need to provide the application name (`RELATIONALAI`).

#### 🔐 About Authentication

In this tutorial, we’ll use **active_session**. However, other authentication methods—such as **Key Pair Authentication**, **Password** or **OAuth Token Authentication**—are also supported. For more details, refer to the [RelationalAI GNN documentation](GNN_SDK_DOCUMENTATION.md).



In [ ]:
# load the snowflake configuration to a python dict
snowflake_config = {
    "app_name": "RELATIONALAI",
    "auth_method": "active_session"
}


## ⚙️ Managing Your GNN Engines

The `Provider` class in the RelationalAI GNN SDK allows you to manage your GNN engines seamlessly. Below, we walk through common operations you can perform with the `Provider`:

* ✅ Create a new GNN engine
* 📋 List all available engines
* 🔍 Check the status of an engine
* 🔄 Resume a paused engine
* ❌ Delete an engine

Each of these operations can be done with simple method calls, as shown in the following examples.


In [ ]:
from rai_gnns_experimental import ColumnDType
from rai_gnns_experimental import EvaluationMetric
from rai_gnns_experimental import ForeignKey
from rai_gnns_experimental import GNNTable
from rai_gnns_experimental import JobManager
from rai_gnns_experimental import LinkTask
from rai_gnns_experimental import OutputConfig
from rai_gnns_experimental import Provider
from rai_gnns_experimental import SnowflakeConnector
from rai_gnns_experimental import TaskType
from rai_gnns_experimental import Trainer
from rai_gnns_experimental import TrainerConfig



import time

In [ ]:
# engine setup
engine_name = "my_engine"
engine_size = "GPU_NV_S" # Available sizes: "GPU_NV_S" or "HIGHMEM_X64_S"

# database /s chema
DB_NAME = "TF_DB"
SCHEMA_NAME = "TF_SCHEMA"
DB_SCHEMA = DB_NAME+"."+SCHEMA_NAME


# dataset name
DATASET_NAME = "tokenflow"

# buyer data
BUYER_SOURCE_TABLE = DB_SCHEMA+".BUYERS"
BUYER_NAME = "buyers"
BUYER_PRIMARY_KEY = "BUY_TOKEN_ADDRESS"

# sender data
SENDER_SOURCE_TABLE = DB_SCHEMA+".SENDERS"
SENDER_NAME = "senders"
SENDER_PRIMARY_KEY = "TX_SENDER_ADDRESS"

# transaction data
TRANSACTION_SOURCE_TABLE = DB_SCHEMA+".TRANSACTIONS"
TRANSACTION_NAME = "transactions"
TIME_COLUMN = "BLOCK_TIMESTAMP"

# link task
LINK_TASK_NAME = "sender_buyer"
TASK_TRAIN_TABLE = DB_SCHEMA+".TRAIN"
TASK_TEST_TABLE  = DB_SCHEMA+".TEST"
TASK_VALIDATION_TABLE = DB_SCHEMA+".VALIDATION"

# model params
MODEL_DEVICE = "cuda" # either 'cuda' or 'cpu'
MODEL_N_EPOCHS = 3
MODEL_MAX_ITERS = 200
MODEL_TEXT_EMBEDDER = "model2vec-potion-base-4M"


OUTPUT_ALIAS = "TEST_PREDS_1"
TEST_BATCH_SIZE = 128

In [ ]:
# initialize the provider using the snowflake configuration
# (note: you might be prompted from your MFA app at this point)
provider = Provider(**snowflake_config)



In [ ]:
# We can create a new GNN engine, currently two
# engine types are supported "GPU_NV_S" and "HIGHMEM_X64_S"
# here we create an engine by specifying the engine name (a
# custom name) and also the type of the engine, specified by the
# "size" parameter
# Available sizes: "GPU_NV_S" or "HIGHMEM_X64_S"
#
# provider.create_gnn(
#    name=engine_name,
#    size="GPU_NV_S"  # or "HIGHMEM_X64_S"
#)


# check if engine exists, if yes, resume if not create it
if not provider.get_gnn(engine_name): 
    print(f"Creating Engine {engine_name}")
    provider.create_gnn(name=engine_name, size=engine_size)
else:
    print(f"Resuming Engine {engine_name}")
    provider.resume_gnn(name=engine_name)

In [ ]:
# HINT: We can always resume a GNN engine that has been suspended:
# Note: Engine provisioning can take some minutes. Please
# check the engine status using provider.get_gnn(name="my_engine")

# provider.resume_gnn(name=engine_name)

# And if we need we can also delete a GNN engine

# provider.delete_gnn(name=engine_name)

# we can also check the existing engines. If there is not engines listed here you would need to create one using provider.create_gnn(name="my_engine", size="GPU_NV_S")
# provider.list_gnns()


In [ ]:


# If we resume an engine, we can directly see the status of the engine
# the status of the engine 'READY' marks the fact that the
# engine is ready to be used. A `PENDING` status marks the
# fact that the engine has  been automaticaly suspended.
# Notice also that under the settings 
# the provider exposes a URL for the MLFLOW endpoint
# that we can use to track our experiments
# NOTE: we should wait until the status is READY

engine_data = provider.get_gnn(engine_name) 
while not engine_data or engine_data["state"] != 'READY':
    time.sleep(10)
    engine_data = provider.get_gnn(engine_name) 
    

print(f'ENGINE {engine_name} READY')

⚠️ **Warning:** To  make sure to check the engine status (e.g., via `provider.get_gnn("engine_name")`) and make sure that the status is `READY` 


## 🔌 Connector Setup

The `Connector` class, like the `Provider` class, is used to communicate with Snowflake. However, while the `Provider` is responsible for managing GNN engines, the `Connector` is specifically used to interface with the **GNN learning engine** itself.

You’ll use the `Connector` instance as an input to all SDK components that need to send requests to the GNN engine—such as loading data, running training jobs, or performing inference.

In short:

* `Provider` → Manages GNN engine instances (create, list, delete, etc.)
* `Connector` → Sends requests to a specific GNN engine for processing tasks

Let’s now walk through how to create and use a `Connector`.


In [ ]:
# we initialize the connector and passing all our credentials.
connector = SnowflakeConnector(
    **snowflake_config,
    engine_name=engine_name,
)
# the connector also provides access to MLFLOW that you can
# use to monitor your experiments and register trained GNN models
# connector.mlflow_session_url

## 📊 Preparing the data: Creating the GNN tables

In this section, we will define the GNN tables and the associated learning task. These components will then be used to construct a GNN dataset suitable for training.

For this tutorial, we’ll use the Tokenflow database as our working example. This database includes three tables: BUYERS, SENDERS, and TRANSACTIONS. The TRANSACTIONS table links BUYERS to SENDERS. Our objective is to predict which BUYERS a given SENDER is most likely to transact with in the future—framing this as a link prediction task.

Before loading the data, ensure that your application has the necessary permissions to access the relevant tables. In Snowflake, this can be achieved by running the following commands:

```sql
-- here we grant access to all schemas and tables, you might want to
-- select specific tables and schemas to grant access to
GRANT USAGE ON DATABASE GNN_TOKENFLOW TO APPLICATION RELATIONALAI;
GRANT USAGE ON ALL SCHEMAS IN DATABASE GNN_TOKENFLOW TO APPLICATION RELATIONALAI;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN DATABASE GNN_TOKENFLOW TO APPLICATION RELATIONALAI;
-- grant write access to write results, we encourage the user to select specific schemas
-- to give write access to
GRANT CREATE TABLE ON ALL SCHEMAS IN DATABASE GNN_TOKENFLOW TO APPLICATION RELATIONALAI;
```

In [ ]:
# create a table for the buyers and set the 
# BUY_TOKEN_ADDRESS as a primary key (primary and 
# foreign keys are used to construct the edges of the graph)
buyers_table = GNNTable(
    connector=connector,
    source=BUYER_SOURCE_TABLE,
    name=BUYER_NAME,
    primary_key=BUYER_PRIMARY_KEY,
)
buyers_table.show_table()


In [ ]:
# in a similar manner we can create the SENDERS table
senders_table = GNNTable(
    connector=connector,
    source=SENDER_SOURCE_TABLE,
    name=SENDER_NAME,
    primary_key=SENDER_PRIMARY_KEY,
)
senders_table.show_table()

In [ ]:
# and finally we will link the two tables using the foreign
# keys from the TRANSACTIONS table. Note: the transactions
# table  has also one special "time column" that will be used
# to prevent data leakage (see the documentation for more details)
transactions_table = GNNTable(
    connector=connector,
    source=TRANSACTION_SOURCE_TABLE,
    name=TRANSACTION_NAME,
    foreign_keys=[
        ForeignKey(
            column_name=SENDER_PRIMARY_KEY, link_to=SENDER_NAME+"."+SENDER_PRIMARY_KEY
        ),
        ForeignKey(column_name=BUYER_PRIMARY_KEY, link_to=BUYER_NAME+"."+BUYER_PRIMARY_KEY),
    ],
    time_column=TIME_COLUMN,
)
# modify column dtypes if needed
transactions_table.update_column_dtype(
    col_name="BUY_AMOUNT", dtype=ColumnDType.integer_t
)
transactions_table.update_column_dtype(
    col_name="SELL_AMOUNT", dtype=ColumnDType.integer_t
)
transactions_table.show_table()


## 🔧 Preparing the Data: Creating the Task

To define the task, we begin by specifying the locations of the training, validation, and test datasets. We also identify the source and destination entity tables, along with the corresponding columns that uniquely identify each entity.

Since this is a **link prediction** task, our objective is to predict future connections between a source entity and a destination entity.

Additionally, we define a **timestamp column** to avoid information leakage by ensuring that future data doesn't influence past predictions (see [documentation](GNN_SDK_DOCUMENTATION.md) for details). Lastly, we specify the **evaluation metric**—in this case, **Mean Average Precision (MAP)**—to assess the performance of the model.


In [ ]:
link_pred_task = LinkTask(
    connector=connector,
    name=LINK_TASK_NAME,
    task_data_source={
        "train": TASK_TRAIN_TABLE, 
        "test": TASK_TEST_TABLE, 
        "validation": TASK_VALIDATION_TABLE
    },
    # name of source entity column that we want to do predictions for
    source_entity_column=SENDER_PRIMARY_KEY,
    # name of GNN table that column is at
    source_entity_table=SENDER_NAME,
    # name of target entity column that we want to predict
    target_entity_column=BUYER_PRIMARY_KEY,
    # name of GNN table that column is at
    target_entity_table=BUYER_NAME,
    time_column=TIME_COLUMN,
    task_type=TaskType.LINK_PREDICTION,
    evaluation_metric=EvaluationMetric(name="link_prediction_map", eval_at_k=12),
)

link_pred_task.show_task()

## 🧩 Preparing the Data: Creating the Dataset

Finally, we combine all the components by constructing a dataset object that encapsulates both the GNN tables and the task definition. This dataset will serve as the input to the model training pipeline, ensuring that the task and its associated data are tightly integrated and ready for downstream processing.


In [ ]:
from rai_gnns_experimental import Dataset

dataset = Dataset(
    connector=connector,
    dataset_name=DATASET_NAME,
    tables=[buyers_table, transactions_table, senders_table],
    task_description=link_pred_task,
)

In [ ]:
# we can also visualize the dataset 
from graphviz import Source
graph = dataset.visualize_dataset(show_dtypes=True)
# play with font size and plot size to get a good visualization
for node in graph.get_nodes():    
    font_size = node.get_attributes()['fontsize']
    font_size = "16"
    node.set('fontsize', font_size)
    

graph.set_graph_defaults(size='"10,10!"')

src = Source(graph.to_string())
src  # Display in notebo

## 🚀 GNN Model Training

Now that our dataset is ready, we can train our first GNN model. We’ll begin by defining a **configuration** that specifies the training parameters, such as model architecture, optimizer settings, and training duration.

Next, we’ll instantiate a **trainer** using this configuration. The trainer will consume the dataset we previously created and manage the entire training process. By calling the `fit()` method on the trainer, we initiate a training job—whose progress and status can be monitored throughout execution.



In [ ]:
# the first step will be to define a configuration for our Trainer.
# the configuration includes many parameters that are explained in
# detail in the documentation. It does not only provide parameters
# for the graph neural network but also parameters for other components
# of the model (such as feature extractors, prediction head parameters,
# training parameters etc.)
model_config = TrainerConfig(
    connector=connector,
    device=MODEL_DEVICE,  # either 'cuda' or 'cpu'
    n_epochs=MODEL_N_EPOCHS,
    max_iters=MODEL_MAX_ITERS,
    text_embedder=MODEL_TEXT_EMBEDDER,
)

In [ ]:
# we initialize now our trainer object with the trainer configuration
# the trainer object can be used to train a model, to perform inference
# or to perform training & inference.
trainer = Trainer(connector=connector, config=model_config)

In [ ]:
# in our first example we will use the trainer to perform training only.
# every time the trainer is "executed" (calling fit(), predict() or fit_predict())
# it returns a job object that can be used to monitor the current running job.
# See the documentation for the meaning of the job statuses
train_job = trainer.fit(dataset=dataset)

In [ ]:
# now we can monitor the job status
# observe that once the job is running we also get back an experiment name
# we will see later how we can use that to perform inference
train_job.get_status()

In [ ]:
# we can also stream the logs of the training job in real time
# Hint: You can stop the cell execution to stop monitoring of logs
# Hint: At this point you can also open the MLFLow URL to monitor your experiments
train_job.stream_logs()

In [ ]:
# HINT: one can cancel a running job as well
# train_job.cancel()

In [ ]:
# it might be the case that we have lost track of
# the jobs that we are running. To this end we also
# provide to the user a JobManager object that can
# give us the status of all current jobs. Let's see
# an example:
job_manager = JobManager(connector=connector)
job_manager.show_jobs()

In [ ]:
# we can also get back job details using it's job id
# foo_job = job_manager.fetch_job("01bd4f74-0000-21eb-001d-cf8b00125bc2")
# hint: the job manager can be used to cancel any job as well

## 🔍 Inference Using a Trained Model

Finally, we’ll demonstrate how to perform inference using the model we’ve just trained. In this example, we'll directly use the recently trained model to generate predictions.

For more advanced use cases—such as registering a model for reuse or automatically selecting the best-performing model—please refer to the [documentation](GNN_SDK_DOCUMENTATION.md) for further guidance.


In [ ]:
output_config = OutputConfig.snowflake(database_name=DB_NAME, schema_name=SCHEMA_NAME)
# make sure that the table with the same alias does not already exist
# we never overwrite tables
inference_job = trainer.predict(
    output_alias=OUTPUT_ALIAS,
    output_config=output_config,
    test_batch_size=TEST_BATCH_SIZE,
    dataset=dataset,
    model_run_id=train_job.model_run_id,
    extract_embeddings=True,
)


In [ ]:
inference_job.get_status()